# Question Generation

> Question Generation: Here we will put together classes / methods that provides Question Generation workflow for our passages

In [1]:
#| default_exp question_generation

In [1]:
#| hide
from nbdev.showdoc import *

In [332]:
#| hide
import nbdev; nbdev.nbdev_export()

## Imports

In [3]:
#| export
from loguru import logger
import os
from pathlib import Path
from fastcore.basics import patch_to, patch
from typing import Union, List

from zeroqaret.dataset import BEIRDataset, our_list as eval_list
from zeroqaret.helper import get_today, create_header

import pandas as pd

import torch
from torch import Tensor

import textwrap

from colbert.modeling.colbert import ColBERT
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

from tqdm import tqdm

/home/bengsoon/conda/envs/xcs224/lib/python3.9/site-packages/beir/datasets/data_loader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [331]:
#| export
from langchain.llms import Ollama
from langchain.embeddings import OllamaEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 
from langchain.schema import OutputParserException

from typing import Dict

In [5]:
#| export
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [6]:
#| export
from sentence_transformers import SentenceTransformer, util
import random

## Dataset

In [7]:
beir_dataset = BEIRDataset()

2023-10-28 19:09:37.211 | INFO     | zeroqaret.dataset:__init__:51 - Datasets will be saved in '/home/bengsoon/Projects/xcs224u_project/zeroqaret/datasets'


We will load the corpus for `trec-covid` as a start. We will not load the `queries` in "reality", we do not have access to these:

In [8]:
dataset_name = "trec-covid"

# We want to load only the corpus / passages  
raw_corpus, _, _ = beir_dataset.load_dataset(dataset_name)

2023-10-28 19:09:37.238 | INFO     | zeroqaret.dataset:load_dataset:62 - Downloading dataset 'trec-covid'...
2023-10-28 19:09:37.239 | INFO     | zeroqaret.dataset:load_dataset:65 - Saved on '/home/bengsoon/Projects/xcs224u_project/zeroqaret/datasets/trec-covid'


  0%|          | 0/171332 [00:00<?, ?it/s]

Let's convert `trec_corpus` into a list of passages, but first let's map the positional indices of the list to the original dataset's `pid`(s) 

In [9]:
trec_corpus_ids = {idx: str(val) for idx, val in enumerate(list(raw_corpus))}
trec_corpus = [(passage.get("title", "") + " " + passage["text"].strip()).strip() for passage in raw_corpus.values()] 

Let's look at the first 5 samples of the passages

In [8]:
print("\n\n".join(trec_corpus[:5]))

Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35

## LLM

In [10]:
llm_model_name = "mistral:instruct" # for 4-bit q: use `mistral:instruct`. for 8-bit q: use `mistral:7b-instruct-q8_0`.
base_url = "http://localhost:11434"


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = Ollama(base_url=base_url,
             model=llm_model_name, 
             callback_manager = callback_manager)

Let's see if we are connected to our model:

In [55]:
llm("Can you tell me who you are?")

I'm Mistral, a language model trained by the Mistral AI team.

"I'm Mistral, a language model trained by the Mistral AI team."

In [56]:
llm("What can you do?")

I am unable to perform actions as I do not have the ability to execute commands or interact with the physical world. I can only provide information, answer questions, and engage in text-based conversation.

'I am unable to perform actions as I do not have the ability to execute commands or interact with the physical world. I can only provide information, answer questions, and engage in text-based conversation.'

> Great! Now let's create a pipeline for question generation

## Question Generation with `Mistral-7B-4q` & Round-trip Consistency Check

#### Generate Question (QG)

In [66]:
# Question Generation template
qg_prompt_template = """<s>[INST]
You are a curious person who loves to ask pertinent questions. Given the Passage below, it is your job to give a correct highly descriptive title, ask the relevant right question and correct one-sentenced short answer strictly from the given passage.
----
{format_instructions}
---- 
Passage: {passage}
----
Title:
Question: 
Answer:
[/INST]"""

Let's create `response_schemas` so that we can instruct the model to output in a specific JSON format:

In [67]:
# create response schemas
qg_response_schemas = [
    # ResponseSchema(name="passage", description="Repeat of the input passage"), 
    ResponseSchema(name="title", description="Descriptive generated title based on the passage"),
    ResponseSchema(name="question", description="Relevant generated question based on the passage"),
    ResponseSchema(name="answer", description="Generated one-sentenced short answer based on the generated question and passage"),

]

# create an output parser
qg_output_parser = StructuredOutputParser.from_response_schemas(qg_response_schemas)

# get format instructions to enforce the expected json format
qg_format_instructions = qg_output_parser.get_format_instructions()

In [68]:
print(qg_format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"title": string  // Descriptive generated title based on the passage
	"question": string  // Relevant generated question based on the passage
	"answer": string  // Generated one-sentenced short answer based on the generated question and passage
}
```


In [69]:
qg_prompt = PromptTemplate(
    template=qg_prompt_template,
    input_variables=["passage"],
    partial_variables={"format_instructions": qg_format_instructions}
)

In [70]:
print(qg_prompt.format_prompt(passage=trec_corpus[45]).text)

<s>[INST]
You are a curious person who loves to ask pertinent questions. Given the Passage below, it is your job to give a correct highly descriptive title, ask the relevant right question and correct one-sentenced short answer strictly from the given passage.
----
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"title": string  // Descriptive generated title based on the passage
	"question": string  // Relevant generated question based on the passage
	"answer": string  // Generated one-sentenced short answer based on the generated question and passage
}
```
---- 
Passage: Sequence specific visual detection of LAMP reactions by addition of cationic polymers BACKGROUND: Development of a practical gene point-of-care testing device (g-POCT device) requires innovative detection methods for demonstrating the results of the gene amplification reaction without the use of expensive equipment. We

In [71]:
qg_chain = qg_prompt | llm

In [72]:
qg_output_trec_45 = llm(qg_prompt.format_prompt(passage=trec_corpus[45]).text, seed=158)

```json
{
	"title": "Development of a Gene Point-of-Care Testing Device",
	"question": "What is the method for the sequence-specific visual detection of minute amounts of nucleic acids in Loop mediated isothermal Amplification (LAMP) reactions?",
	"answer": "Oligo DNA probes labeled with different fluorescent dyes are prepared for multiple nucleic acid templates. At completion of the LAMP reaction, an optimal amount of low molecular weight polyethylenimine (PEI) is added, resulting in the precipitation of the insoluble LAMP amplicon-PEI complex. The fluorescently labeled Oligo DNA probes hybridize to the LAMP product and the precipitate emits fluorescence corresponding to the amplified nucleic acid templates. The color of emitted fluorescence can be detected easily by naked eye on a conventional UV illuminator."
}
```

In [74]:
parsed_output = qg_output_parser.parse(qg_output_trec_45)

In [75]:
parsed_output["answer"]

'Oligo DNA probes labeled with different fluorescent dyes are prepared for multiple nucleic acid templates. At completion of the LAMP reaction, an optimal amount of low molecular weight polyethylenimine (PEI) is added, resulting in the precipitation of the insoluble LAMP amplicon-PEI complex. The fluorescently labeled Oligo DNA probes hybridize to the LAMP product and the precipitate emits fluorescence corresponding to the amplified nucleic acid templates. The color of emitted fluorescence can be detected easily by naked eye on a conventional UV illuminator.'

#### Check Answer

In [76]:
check_answer_prompt_template = """<s>[INST]
You are an expert on the topic in the passage below. Given the Title, Passage and Question below, it is your job to provide a correct and relevant one-sentenced short answer.
----
{format_instructions}
----
Passage: {passage}
Title: {title}
Question: {question}
---- 
Answer:
[/INST]"""

In [77]:
# create response schemas
check_answer_response_schemas = [
    # ResponseSchema(name="passage", description="Repeat of the input passage"), 
    ResponseSchema(name="answer", description="Generated one-sentenced short answer based on the title, question and passage"),
]

# create an output parser
check_answer_output_parser = StructuredOutputParser.from_response_schemas(check_answer_response_schemas)

# get format instructions to enforce the expected json format
check_answer_format_instructions = check_answer_output_parser.get_format_instructions()

In [78]:
check_answer_prompt = PromptTemplate(
    template=check_answer_prompt_template,
    input_variables=["passage", "title", "question"],
    partial_variables={"format_instructions": check_answer_format_instructions}
)

In [79]:
print(check_answer_prompt.format_prompt(passage=trec_corpus[45], title=parsed_output["title"], question=parsed_output["question"]).text)

<s>[INST]
You are an expert on the topic in the passage below. Given the Title, Passage and Question below, it is your job to provide a correct and relevant one-sentenced short answer.
----
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // Generated one-sentenced short answer based on the title, question and passage
}
```
----
Passage: Sequence specific visual detection of LAMP reactions by addition of cationic polymers BACKGROUND: Development of a practical gene point-of-care testing device (g-POCT device) requires innovative detection methods for demonstrating the results of the gene amplification reaction without the use of expensive equipment. We have studied a new method for the sequence-specific visual detection of minute amounts of nucleic acids using precipitation reaction by addition of cationic polymers to amplicons of Loop mediated isothermal Amplification (

In [357]:
check_answer_chain = check_answer_prompt | llm

In [376]:
check_answer_output_trec_45 = llm(check_answer_prompt.format_prompt(passage=trec_corpus[45], title=parsed_output["title"], question=parsed_output["question"]).text, seed=158)

```json
{
	"answer": "The researchers detected the presence or absence of minute amounts of nucleic acid templates by visual assessment of the color of the LAMP amplicon-PEI complex precipitate."
}
```

In [358]:
check_answer_output_trec_45 = check_answer_chain.invoke({"passage": trec_corpus[45], "title": parsed_output["title"], "question": parsed_output["question"]})

```json
{
    "answer": "The purpose of this study is to develop a practical gene point-of-care testing device using fluorescent labeled oligo DNA probes for detecting the presence or absence of minute amounts of nucleic acid templates in a simple manner through visual assessment."
}
```

In [377]:
check_answer_parsed_output = check_answer_output_parser.parse(check_answer_output_trec_45)

In [378]:
print(f"QG Answer:\n\t {parsed_output['answer']} \n" + "-"*50 + f"\n CA Answer:\n\t {check_answer_parsed_output['answer']}")

QG Answer:
	 The researchers detected the presence or absence of minute amounts of nucleic acid templates by visual assessment for the color of the LAMP amplicon-PEI complex precipitate. 
--------------------------------------------------
 CA Answer:
	 The researchers detected the presence or absence of minute amounts of nucleic acid templates by visual assessment of the color of the LAMP amplicon-PEI complex precipitate.


#### Find similarity

In [402]:
sbert_model = SentenceTransformer('all-mpnet-base-v2')

In [403]:
qg_answer_emb = sbert_model.encode(parsed_output["answer"])
ca_answer_emb = sbert_model.encode(check_answer_parsed_output["answer"])

In [404]:
cosine_scores = util.cos_sim(qg_answer_emb, ca_answer_emb).to("cpu")

In [405]:
cosine_scores

tensor([[0.9988]])

In [406]:
oembed = OllamaEmbeddings(base_url=base_url, model=llm_model_name)

In [407]:
qg_answer_oemb = oembed.embed_query(parsed_output["answer"])
ca_answer_oemb = oembed.embed_query(check_answer_parsed_output["answer"])

In [408]:
cosine_scores_oemb = util.cos_sim(qg_answer_oemb, ca_answer_oemb)

In [409]:
cosine_scores_oemb

tensor([[0.9963]])

### Putting It Altogether (Question Generator and Round Trip Consistency)

In [303]:
#| export
class QuestionGenerator:

    def __init__(self, 
                 ollama_base_url: str = 'http://localhost:11434',
                 ollama_model_name: str = 'mistral:instruct',
                 random_seed: int = 158
                ):


        self.ollama_base_url = ollama_base_url
        self.ollama_model_name = ollama_model_name
        self.random_seed = random_seed
        
        # callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        
        self.llm = Ollama(
            base_url=ollama_base_url,
            model=ollama_model_name, 
                )
        
        self._setup_question_generator()
        self._setup_answer_checker()

    def _setup_question_generator(self,
                                 qg_prompt_template: str = None, # LLM Prompt template
                                ) -> None:
        " Sets up all the shared variables and methods for question generator "

        
        # Question Generation template
        self.qg_prompt_template = qg_prompt_template or """
<s>[INST]
You are a curious person who loves to ask pertinent questions. 
Given the Passage below, it is your job to give a correct highly descriptive title, ask the relevant right question and correct one-sentenced short answer strictly from the given passage.
----
{format_instructions}
---- 
Passage: {passage}
----
Title:
Question: 
Answer:
[/INST]"""

        # create response schemas
        self.qg_response_schemas = [
            # ResponseSchema(name="passage", description="Repeat of the input passage"), 
            ResponseSchema(name="title", description="Descriptive generated title based on the passage"),
            ResponseSchema(name="question", description="Relevant generated question based on the passage"),
            ResponseSchema(name="answer", description="Generated one-sentenced short answer based on the generated question and passage"),
        ]
        
        # create an output parser
        self.qg_output_parser = StructuredOutputParser.from_response_schemas(self.qg_response_schemas)
        
        # get format instructions to enforce the expected json format
        self.qg_format_instructions = self.qg_output_parser.get_format_instructions()

        # prompt template
        self.qg_prompt = PromptTemplate(
                                template=self.qg_prompt_template,
                                input_variables=["passage"],
                                partial_variables={"format_instructions": self.qg_format_instructions}
                            )

    def _setup_answer_checker(self,
                              ac_prompt_template: str = None, # LLM Prompt template
                             ) -> None:
        " Sets up all the shared variables and methods for the Answer Checker "
        
        self.answer_checker_prompt_template = """
<s>[INST]
You are an expert on the topic in the passage below. Given the Title, Passage and Question below, it is your job to provide a correct and relevant one-sentenced short answer.
----
{format_instructions}
----
Passage: {passage}
Title: {title}
Question: {question}
---- 
Answer:
[/INST]
"""

        # create response schemas
        self.answer_checker_response_schemas = [
            ResponseSchema(name="answer", description="Generated one-sentenced short answer based on the title, question and passage"),
        ]
        
        # create an output parser
        self.answer_checker_output_parser = StructuredOutputParser.from_response_schemas(self.answer_checker_response_schemas)
        
        # get format instructions to enforce the expected json format
        self.answer_checker_format_instructions = self.answer_checker_output_parser.get_format_instructions()

        # answer checker prompt
        self.answer_checker_prompt = PromptTemplate(
                template=self.answer_checker_prompt_template,
                input_variables=["passage", "title", "question"],
                partial_variables={"format_instructions": self.answer_checker_format_instructions}
            )

    def generate_question(self,
                          passage: str, # passage
                          random_seed: int = None, # if provided, it will replace random_seed
                          verbose: bool = False, # prints prompt
                          failure_count: int = None, # current count of failure to generate 
                          max_generation_tries: int = 10, # if failure_count == max_generation_tries, it will return the llm response params as empty strings 
                           **kwargs: "Any",
                         ) -> Dict[str, str]:
        """ 
        Prompts LLM to generate title, question and answer given `passage`. 
        Returns {"title": generated title, "question": generated question, "answer": generated answer }
        """ 

        random_seed=random_seed or self.random_seed
        
        prompt = self.qg_prompt.format_prompt(passage=passage).text
        if verbose in ["all"]: 
            logger.info("\n" + f"Question Generation Prompt:".center(100) + "\n\n" + textwrap.fill(prompt, 100))

        #####  The code block below checks to see if:
        #####      - the parser could parse the response from LLM
        #####      - the parsed results contains all the required params 
        # Inherit failure_count input, or start from 0 if it was passed
        if not failure_count:
            failure_count = 0
            
        success = False
        while success == False:
            logger.info(f"Current Question Generation failure_count: {failure_count}")
            try:
                res = self.llm(prompt, seed=random_seed, **kwargs)
                res = self.qg_output_parser.parse(res)
                assert self._output_is_dict(res), "Results were not parsed as dict!"
                assert self._check_params_in_output(res, ["title", "question", "answer"]), "Results does not contain all of ('title', 'question', 'answer')!"
                success = True
                
            except OutputParserException as e:
                logger.warning(f"Unable to parse results. Exception msg: {e}")
                failure_count += 1
                logger.info(f"Current failure count: {failure_count}")
                
            except AssertionError as e:
                logger.warning(e)
                failure_count += 1
                logger.info(f"Current failure count: {failure_count}")
                
            if failure_count >= max_generation_tries:
                logger.warning(f"#Error: Question Generation failure_count > max_generation_tries '{max_generation_tries}'." + "\{'title': '', 'question': '', 'answer': ''\}...")
                res = {"title": "", "question": "", "answer": ""}
                break

        for key, val in res.items():
            if type(val) == list:
                res[key] = " ".join(val)
                
        if verbose in ["all"]:
            logger.info("\n" + f"Question Generation Results:".center(100) + "\n\n" + textwrap.fill(str(res), 100))
        return res

    def generate_answer_to_check(self,
                                 passage: str, # passage
                                 title: str, # title
                                 question: str, # question
                                 random_seed: int = None, # if provided, it will replace random_seed
                                 verbose: bool = False, # prints prompt
                                 failure_count: int = None, # current count of failure to generate 
                                 max_generation_tries: int = 10, # if failure_count == max_generation_tries, it will return the llm response params as empty strings 
                                 **kwargs: "Any",
                                  ) -> Dict[str, str]:
        """ 
        Prompts LLM to answer given `passage`, `title`, `question`.
        Returns {"answer": generated answer }
        """ 

        random_seed=random_seed or self.random_seed
        
        prompt = self.answer_checker_prompt.format_prompt(passage=passage, title=title, question=question).text
        if verbose in ["all"]: 
            logger.info("\n" + f"Answer Checker Prompt:".center(100) + "\n\n" + textwrap.fill(prompt, 100))

        #####  The code block below checks to see if:
        #####      - the parser could parse the response from LLM
        #####      - the parsed results contains all the required params 
        # Inherit failure_count input, or start from 0 if it was passed
        if not failure_count:
            failure_count = 0
            
        success = False
        while success == False:
            logger.info(f"Current Question Generation failure_count: {failure_count}")
            try:
                res = self.llm(prompt, seed=random_seed, **kwargs)
                res = self.answer_checker_output_parser.parse(res)
                assert self._output_is_dict(res), "Results were not parsed as dict!"
                assert self._check_params_in_output(res, ["answer"]), "Results does not contain all of ('answer')!"
                success = True
                
            except OutputParserException as e:
                logger.warning(f"#Error: Unable to parse results. Exception msg: {e}")
                failure_count += 1
                logger.info(f"Current failure count: {failure_count}")
                
            except AssertionError as e:
                logger.warning(f"#Error: {e}")
                failure_count += 1
                logger.info(f"Current failure count: {failure_count}")
                
            if failure_count >= max_generation_tries:
                logger.warning(f"#Error: Answer Checker failure_count > max_generation_tries '{max_generation_tries}'." + "\{'answer': ''\}...")
                res = {"answer": ""}
                break

        for key, val in res.items():
            if type(val) == list:
                res[key] = " ".join(val)
        
        if verbose in ["all"]:
            logger.info("\n" + f"Answer Checker Results:".center(100) + "\n\n" + textwrap.fill(str(res), 100))
            
        return res

    def _output_is_dict(self,
                        res: Union[str, dict], # results from llm
                      ) -> bool:
        " Checks to see if output is dict. "

        return type(res) == dict

    def _check_params_in_output(self,
                                res: Union[str, dict], # results from llm
                                params: List, # list of keys (parameters) to test
                               ) -> bool:
        " Checks to see if res contains all the params as keys "
        
        return all([param in res for param in params])

    def round_trip_question_generation(self,
                                       passage: str, # passage
                                       embedding_model: str = 'all-MiniLM-L6-v2', # embedding model: any SBERT emb models or 'llm' if use emb from Ollama's llm
                                       cos_sim_cutoff: float = 0.8, # cosine-sim cutoff score to accept [-1, 1]
                                       random_seed: int = None, # if provided, it will replace random_seed
                                       max_generation_tries: int = 10, # if failure_count == max_generation_tries, it will return the llm response params as empty strings 
                                       verbose: str = None, # "all" to report everything, "results" to report only results. 
                                      ) -> Union[Dict[str, str], bool]:
        """ Prompts LLM to generate title, question and answer given `passage` and performs round-trip consistency check.
            Returns {"title": generated title, "question": generated question, "answer": generated answer} if passed, None if rejected.
            Note: We have assigned the generated_results as part of the instance variable of this class for debugging purposes when we loop through the corpus. """
        self.rt_random_seed = random_seed or self.random_seed
        
        if not hasattr(self, "emb_model"):
            logger.info("Setting up embedding model")
            if embedding_model == "llm":
                self.emb_model = OllamaEmbeddings(base_url=base_url, model=llm_model_name).embed_query
            else:
                self.emb_model = SentenceTransformer(embedding_model).encode
            
        logger.info("Generating question...")
        self.generated_results = self.generate_question(passage, verbose=verbose, random_seed=self.rt_random_seed, max_generation_tries=max_generation_tries)

        for key, val in self.generated_results.items():
            if type(val) == list:
                self.generated_results[key] = " ".join(val)
        
        logger.info("Performing round-trip consistency check...")
        self.checker_results = self.generate_answer_to_check(passage, self.generated_results["title"], self.generated_results["question"], verbose=verbose, random_seed=self.rt_random_seed, max_generation_tries=max_generation_tries)

        for key, val in self.checker_results.items():
            if type(val) == list:
                self.checker_results[key] = " ".join(val)
            
        if verbose in ["all", "results"]:
            logger.info("\n" + "." * 100 + "\n" + " Generated Answer: ".center(100, " ") + "\n\n" + textwrap.fill(f"{self.generated_results['answer']}", 100) + "\n\n" +
                        "\n" + " Checker's Answer: ".center(100, " ") + "\n\n" + textwrap.fill(f"{self.checker_results['answer']}", 100) + "\n\n" + "."*100)

        logger.info("Performing similarity calculation ...")
        gen_a_emb = self.emb_model(str(self.generated_results["answer"])) # force it to be string
        checker_a_emb = self.emb_model(str(self.checker_results["answer"])) # force it to be string

        score = self.cos_sim(gen_a_emb, checker_a_emb)
        if verbose in ["all", "results"]: 
            logger.info(f"Score: {score}")

        if score < cos_sim_cutoff:
            logger.info(f"Rejecting generated question set as it failed the consistency check")
            return None
        else:
            logger.info(f"Passed consistency check.")
            return self.generated_results        
            
        
    def cos_sim(self, a: Tensor, b: Tensor):
        """
        Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
        
        :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
        Adapted from https://github.com/UKPLab/sentence-transformers/blob/c5f93f70eca933c78695c5bc686ceda59651ae3b/sentence_transformers/util.py
        """
        if not isinstance(a, torch.Tensor):
            a = torch.tensor(a)
    
        if not isinstance(b, torch.Tensor):
            b = torch.tensor(b)
    
        if len(a.shape) == 1:
            a = a.unsqueeze(0)
    
        if len(b.shape) == 1:
            b = b.unsqueeze(0)
    
        a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
        b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
        return torch.mm(a_norm, b_norm.transpose(0, 1)).to('cpu')

In [304]:
question_generator = QuestionGenerator()

In [329]:
#| export
@patch_to(QuestionGenerator)
def generate_questions_corpus(self,
                              dataset_name: str, # name of the dataset
                              corpus: List,
                              corpus_ids: Dict,
                              df_checkpoint_path: str, # (csv format) checkpoint to save csv 
                              mode: str = "resume", # ["resume", "replace"] - resume if continue from checkpoint, replace if generation from scratch
                              cos_sim_cutoff: float = 0.8, # cosine-sim cutoff score to accept [-1, 1]
                              max_roundtrip_tries: int = 10, # maximum number of roundtrip question generation trials allowed for each document
                              max_generation_tries: int = 10, # maximum number of generation trials allowed independently for QG and Answer-Checker for each document
                              verbose: str = None, # "file" if log only to file w/o stdout (full verbosity), "all" if full verbosity, "results" if only results, "disable" will temporarily disable system logging.
                             ) -> None:
    assert verbose in [None, "all", "results", "disable", "file"], 'verbose options are only [None, "all", "results", "disable", "file"]'
    # disable logger if asked to
    if verbose == "disable": 
        logger.disable("__main__")
    else:
        logger.enable("__main__")

    # make checkpoint path directory if it does not exist:
    Path(df_checkpoint_path).parents[0].mkdir(exist_ok=True)
    
    if verbose == "file":
        logger.configure(handlers=[dict(sink=f"./{dataset_name}_qg_log.log", enqueue=True,format="[{time}] {message}", rotation=1e+7)]) # 100 MB
        verbose = "all" # set full verbosity logging to file
        
    if mode == "resume":
        # This is when we are resuming question generation from previous checkpoint.
        logger.info(f"Resuming question generation from checkpoint '{df_checkpoint_path}'")  
        try:
            qg_df = pd.read_csv(df_checkpoint_path, index_col=0) 
            qg_df["pid"] = qg_df["pid"].astype(str) # cast pid as str even though it may be just numbers
        except:
            raise Exception(f"Unable to read the checkpoint file '{df_checkpoint_path}', check the df_checkpoint_path or use `mode='replace'` to start from scratch.")
            
        last_checkpoint_idx = qg_df.iloc[-1].name # index corresponding to the key in corpus_ids
        last_checkpoint_pid = qg_df.iloc[-1].pid # pid corresponding to the val in corpus_ids

        # check that corpus_ids's correspond to the checkpoint index and pid
        assert corpus_ids[last_checkpoint_idx] == last_checkpoint_pid, "The corpus_ids \
provided does not tally with the idx: pid from dataframe's saved csv checkpoint. \
Please check corpus_ids input."
        starting_id = last_checkpoint_idx + 1 # we want to start one after our last one
        logger.info(f"Resuming from index {starting_id} / pid {corpus_ids[starting_id]}")
        
    elif mode == "replace":
        # we will start from scratch
        if Path(df_checkpoint_path).exists():
            raise Exception(f"{df_checkpoint_path} already exists! Please make a back up and remove the file before starting from scratch!")
        
        qg_df = pd.DataFrame(columns=["pid", "passage", "title", "question", "answer"])
        starting_id = 0   

    for idx in tqdm(range(starting_id, len(corpus)), "Question Generation Progress: "):
        pid = corpus_ids[idx]
        passage = corpus[idx]
        # logger.info("\n\n" + f"                     {idx} - {pid} ".center(150, "#") + "\n\n")
        logger.info(create_header(f"{idx} - {pid}"))
        logger.info("")
        
        similarity_cutoff = cos_sim_cutoff
        
        random_seed = self.random_seed
        success = False
        failure_count = 0
        while success == False:           
            # Performs roundtrip QG. Returns {"title": generated title, "question": generated question, "answer": generated answer} if passed, 
            #     returns None if rejected.
            res = self.round_trip_question_generation(passage, verbose=verbose, random_seed=random_seed, max_generation_tries=max_generation_tries, cos_sim_cutoff=similarity_cutoff) 

            ###############################   Code block below checks to see if ###############################
            #    - `res` is not None
            #    - `res` is dict 
            #    - `res` has the keys {"title": .., "question": .., "answer": ..}
            try:
                assert res, "Rejected by roundtrip check!"
                assert self._output_is_dict(res), "Results were not parsed as dict!"
                assert self._check_params_in_output(res, ["title", "question", "answer"]), "Results does not contain all of ('title', 'question', 'answer')!"
                success = True 
            except AssertionError as e:
                logger.warning(f"#Error: {e}")
                logger.info(f"Current roundtrip failure count: {failure_count}")
                logger.info("Regenerating another response...")
                failure_count += 1
            except:
                logger.warning("#Error: Error in roundtrip check")
                logger.info(f"Current roundtrip failure count: {failure_count}")
                logger.info("Regenerating another response...")
                failure_count += 1
            
            if max_roundtrip_tries - failure_count == 1:
                if similarity_cutoff > 0.2:
                    similarity_cutoff = similarity_cutoff - 0.1
                    logger.info(f"Setting cos-sim cutoff by 0.1...")
                random_seed = random.randint(200, 1000)
            elif failure_count >= max_roundtrip_tries:
                try:
                    assert self._output_is_dict(self.generated_results), "Results were not parsed as dict!"
                    assert self._check_params_in_output(self.generated_results, ["title", "question", "answer"]), "Results does not contain all of ('title', 'question', 'answer')!"
                    logger.warning(f"Maximum roundtrip tries exceeded '{max_roundtrip_tries}'." + f"Returning previously generated results: {self.generated_results}")
                    res = self.generated_results
                    break
                except:
                    logger.warning(f"Maximum roundtrip tries exceeded '{max_roundtrip_tries}'." + "Returning res = \{'title': '', 'question': '', 'answer': ''\}...")
                    res = {"title": "", "question": "", "answer": ""}
                    break
            else:
                random_seed = random.randint(200, 1000)
                
        res["passage"] = passage
        res["pid"] = pid
    
        qg_df = pd.concat((qg_df, pd.DataFrame(res, index=[idx])))
        logger.info(f"Saving dataframe checkpoint as '{df_checkpoint_path}.'")
        qg_df.to_csv(df_checkpoint_path)
    
    # enable logger back
    if verbose == "disable": logger.enable("__main__")

    logger.success("Question Generation is completed!")

Let's generate questions for `trec-covid` dataset

> We have done until 2822 - 8ztx62z0, but as we can see here, it is going to take a long time to finish (~160 - 240 hours to finish!)
> Let's just skip to QG for `scifact` dataset, which has lesser amount of passages

Let's create a function that pulls all that together too:

In [324]:
#| export 
def generate_questions(dataset_name,
                       mode = "resume",
                       max_roundtrip_tries = 2,
                       max_generation_tries = 5,
                       cos_sim_cutoff = 0.6,
                       verbose = "file"):
    # initialize BeirDataset
    beir_dataset = BEIRDataset()
    
    # We want to load only the corpus / passages  
    raw_corpus, _, _ = beir_dataset.load_dataset(dataset_name)

    # get corpus_ids and corpus
    corpus_ids = {idx: str(val) for idx, val in enumerate(list(raw_corpus))}
    corpus = [(passage.get("title", "") + " " + passage["text"].strip()).strip() for passage in raw_corpus.values()] 

    # initialize QuestionGenerator()
    question_generator = QuestionGenerator()

    question_generator.generate_questions_corpus(dataset_name = dataset_name,
                                                 corpus = corpus, 
                                                 corpus_ids=corpus_ids, 
                                                 df_checkpoint_path=f"../datasets/{dataset_name}/qg/{dataset_name}_qg_all.csv", 
                                                 mode=mode, 
                                                 max_roundtrip_tries = max_roundtrip_tries,
                                                 cos_sim_cutoff = cos_sim_cutoff, 
                                                 max_generation_tries = max_generation_tries,
                                                 verbose=verbose)
                       

## scifact

In [325]:
generate_questions(dataset_name="scifact", mode="resume", max_roundtrip_tries=5, max_generation_tries=2, cos_sim_cutoff=0.6, verbose="file")

  0%|          | 0/5183 [00:00<?, ?it/s]

Question Generation Progress: 100%|█████████████████████████████████████████████████████████████████████████████| 5062/5062 [5:38:42<00:00,  4.01s/it]


## nfcorpus

In [328]:
generate_questions(dataset_name="nfcorpus", mode="resume", max_roundtrip_tries=5, max_generation_tries=2, cos_sim_cutoff=0.6, verbose="file")

  0%|          | 0/3633 [00:00<?, ?it/s]

Question Generation Progress: 100%|█████████████████████████████████████████████████████████████████████████████| 2549/2549 [2:51:39<00:00,  4.04s/it]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## 